### TP1: Algoritmos de búsqueda en Torre de Hanoi

Integrantes:
* Mealla Pablo
* Mendoza Dante
* Vasquez Jorge
* Viñas Gustavo



##### 1. ¿Cuáles son los PEAS de este problema? (Performance, Environment, Actuators, Sensors)

**Performance:**  
Resolver correctamente la torre de Hanoi, esto es, que todos los discos queden ordenados en la última pila

**Environment:**  
  * Las tres torres
  * Los discos
  * Las reglas del juego
  * Conocer el estado inicial
  * Conocer el estado objetivo

**Actuators:**  
Movimientos de discos que se realiza en el Juego  
Si fuera manual, sería nuestras manos que realizan el movimiento de discos  
Si fuera una solución por software, sería una función que realiza el movimiento del disco: Programa (node.expand)

**Sensors:**  
Estado actual de los discos (node.state)  
Si fuera manual, serían nuestros ojos que detectan la posición actual de los discos

##### 2. ¿Cuáles son las propiedades del entorno de trabajo?

El entorno de trabajo es
* Es totalmente observable, se pueden observar los discos en todo momento en las diferentes torres.
* Deterministico, porque no hay azar en cambio de estados, y los cambios son decididos por el mismo agente.
* Episódico, porque cada solución no depende de la anterior.
* Estático, el ambiente no cambia mientras el agente esta tomando acciones.
* Discreto, porque con cada movimiento obtengo un estado determinado.
* Agente individual, porque estamos usando un solo agente para la resolución, aunque podría ser multiagente cooperativo si asignaramos ramas a distintos agentes.

##### 3. En el contexto de este problema, establezca cuáles son los: estado, espacio de estados, árbol de búsqueda, nodo de búsqueda, objetivo, acción y frontera.

**Estado:**  
Es la representación de los discos en las pilas para una combinación dada.

**Espacio de estados:**  
Son las posiciones válidas que pueden tomar los discos en las pilas.

**Árbol de búsqueda:**  
Es un diagrama del espacio de estados para encontrar una posible solución al problema de las torres de Hanoi. La raiz representa el estado 
inicial, las ramas representan los movimientos validos, los nodos representan todos los estados posibles y las hojas son los nodos frontera.

**Nodo de búsqueda:**  
Es el estado a partir del cual analizo las acciones que se pueden tomar. Desde este nodo obtengo los siguientes estados posibles.

**Objetivo:**  
Es resolver la torre de Hanoi, es decir, que todos los discos esten ordenados en la última pila.

**Acción:**  
Es el movimiento válido de un disco de una pila hacia otra.

**Frontera:**  
Son los nodos no explorados (estados no explorados), es decir, próximos movimientos posibles aun no realizados.

##### 4. Implemente algún método de búsqueda. Puedes elegir cualquiera menos búsqueda en anchura primero (el desarrollado en clase). Sos libre de elegir cualquiera de los vistos en clases, o inclusive buscar nuevos.


In [1]:
num_disks = 5

In [2]:
# 1. Profundidad primero

from aima_libs.hanoi_states import ProblemHanoi, StatesHanoi
from aima_libs.tree_hanoi import NodeHanoi
from queue import LifoQueue

def depth_first_search(number_disks=5):
    # Inicializamos el problema
    list_disks = [i for i in range(number_disks, 0, -1)]
    initial_state = StatesHanoi(list_disks, [], [], max_disks=number_disks)
    goal_state = StatesHanoi([], [], list_disks, max_disks=number_disks)
    problem = ProblemHanoi(initial=initial_state, goal=goal_state)

    # Creamos una cola LIFO con el nodo inicial
    frontier = LifoQueue()
    frontier.put(NodeHanoi(problem.initial))

    # Creamos el set con estados ya visitados
    explored = set()
    
    node_explored = 0
    
    while not frontier.empty():
        node = frontier.get()
        node_explored += 1
        
        # Agregamos el estado del nodo al set. Esto evita guardar duplicados, porque set nunca tiene elementos repetidos
        explored.add(node.state)
        
        if problem.goal_test(node.state):  # Comprobamos si hemos alcanzado el estado objetivo
            metrics = {
                "solution_found": True,
                "nodes_explored": node_explored,
                "states_visited": len(explored),
                "nodes_in_frontier": frontier.qsize(),
                "max_depth": node.depth,
                "cost_total": node.state.accumulated_cost,
            }
            return node, metrics
        
        # Agregamos a la cola todos los nodos sucesores del nodo actual
        for next_node in node.expand(problem):
            # Solo si el estado del nodo no fue explorado
            if next_node.state not in explored:
                frontier.put(next_node)

    # Si no se encontro la solución, devolvemos la métricas igual
    metrics = {
        "solution_found": False,
        "nodes_explored": node_explored,
        "states_visited": len(explored),
        "nodes_in_frontier": frontier.qsize(),
        "max_depth": node.depth, # OBS: Si no se encontró la solución, este valor solo tiene sentido en breadth_first_search, en otros casos se debe ir llevando registro de cual fue la máxima profundidad
        "cost_total": None,
    }
    return None, metrics

In [3]:
solution_depth_first, metrics = depth_first_search(number_disks=num_disks)
for key, value in metrics.items():
    print(f"{key}: {value}")

solution_found: True
nodes_explored: 122
states_visited: 122
nodes_in_frontier: 63
max_depth: 121
cost_total: 121.0


In [4]:
# 2. Greedy

from aima_libs.hanoi_states import ProblemHanoi, StatesHanoi
from aima_libs.tree_hanoi import NodeHanoi
from aima_libs.aima import PriorityQueue as AimaPriorityQueue

def priority_func(solution_last_rod, node):
    # La heuristica es -1 por disco correctamente ubicado
    last_rod = node.state.get_state()[-1]
    h = 0
    for i in range(0, min(len(last_rod), len(solution_last_rod))):
        if (last_rod[i] != solution_last_rod[i]):
            break
        h -= 1
    return h

def greedy_search(number_disks=5):
    # Inicializamos el problema
    list_disks = [i for i in range(number_disks, 0, -1)]
    initial_state = StatesHanoi(list_disks, [], [], max_disks=number_disks)
    goal_state = StatesHanoi([], [], list_disks, max_disks=number_disks)
    problem = ProblemHanoi(initial=initial_state, goal=goal_state)

    # Creamos una cola priorizada, con funcion heuristica, que contenga el nodo inicial
    solution_last_rod = problem.goal.get_state()[-1]
    frontier = AimaPriorityQueue(order='min', f=lambda x: priority_func(solution_last_rod, x))
    frontier.append(NodeHanoi(problem.initial))

    # Creamos el set con estados ya visitados
    explored = set()
    
    node_explored = 0
    
    while frontier.__len__() != 0:
        _, node = frontier.pop()
        node_explored += 1
        
        # Agregamos el estado del nodo al set. Esto evita guardar duplicados, porque set nunca tiene elementos repetidos
        explored.add(node.state)
        
        if problem.goal_test(node.state):  # Comprobamos si hemos alcanzado el estado objetivo
            metrics = {
                "solution_found": True,
                "nodes_explored": node_explored,
                "states_visited": len(explored),
                "nodes_in_frontier": frontier.__len__(),
                "max_depth": node.depth,
                "cost_total": node.state.accumulated_cost,
            }
            return node, metrics
        
        # Agregamos a la cola todos los nodos sucesores del nodo actual
        for next_node in node.expand(problem):
            # Solo si el estado del nodo no fue explorado
            if next_node.state not in explored:
                frontier.append(next_node)

    # Si no se encontro la solución, devolvemos la métricas igual
    metrics = {
        "solution_found": False,
        "nodes_explored": node_explored,
        "states_visited": len(explored),
        "nodes_in_frontier": frontier.__len__(),
        "max_depth": node.depth, # OBS: Si no se encontró la solución, este valor solo tiene sentido en breadth_first_search, en otros casos se debe ir llevando registro de cual fue la máxima profundidad
        "cost_total": None,
    }
    return None, metrics

In [5]:
solution_greedy, metrics = greedy_search(number_disks=num_disks)
for key, value in metrics.items():
    print(f"{key}: {value}")

solution_found: True
nodes_explored: 160
states_visited: 110
nodes_in_frontier: 46
max_depth: 31
cost_total: 31.0


##### 5. ¿Qué complejidad en tiempo y memoria teórica tiene el algoritmo elegido?

Algoritmo elegido: Greedy

Considerando la siguientes condiciones:

* Cantidad de discos = 5
* Cantidad de Torres = 3
* Una heurística de -1 para cada disco correctamente colocado en la Tercer torre. Es decir que se prioriza los movimientos que maximicen la cantidad de 
discos bien úbicados en la Tercer torre.
* No se permiten estados repetidos.

**Complejidad teórica en tiempo**

Tiempo = Cantidad de estados ó nodos que explora el algoritmo antes de encontrar la solución.

Número total de estados posibles para 5 discos y 3 torres, las combinaciones posibles son:

	3^5 = 243 estados posibles

El mejor caso se da cuando la Heurística siempre elige el movimiento correcto:

* Encuentra la solución rápidamente, explorando solo unos pocos estados.
* Tiempo= Cercano a O(d), donde d es la profundidad de la solución óptima 2^n -1. La cantidad de movimientos ideales sería de 31 movimientos para 5 discos.

    O(d) = O(31)

Peor caso se da cuando:

* Si la heurística no guía bien (es decir, el algoritmo sigue caminos subóptimos):
* El algoritmo podría terminar explorando casi todos los 243 estados posibles antes de dar con la solución.

    O(d)=O(243)


**Complejidad teórica en memoria (nodos almacenados)**

Memoria = cuántos estados guarda el algoritmo mientras busca la solución.
Greedy almacena: 

* Nodos actuales más prometedores (los que tienen mejor heurística).
* Estados visitados para no repetir configuraciones.

Mejor caso (uso optimo de memoria):

* Si la heurística guía bien y encuentra la solución directo:
* Guarda pocos estados, solo los que explora (cerca de 31 nodos).

Memoria mejor caso:
  
    O(31)

Peor caso (memoria):

* Si la heurística falla y explora casi todo:
* Guarda todos los estados posibles (243 estados) para no repetirlos.


Memoria peor caso:
    
    O(243)




##### 6. A nivel implementación, ¿qué tiempo y memoria ocupa el algoritmo? (Se recomienda correr 10 veces y calcular promedio y desvío estándar de las métricas).


In [6]:
import tracemalloc
from math import sqrt
import time

mem_peaks_mb = []
time_measured = []

for _ in range(0, 10):
    tracemalloc.start()
    start_time = time.time()

    depth_first_search(number_disks=num_disks)
    time_measured.append(time.time() - start_time)

    # Para medir memoria consumida usamos el pico de memoria
    _, memory_peak = tracemalloc.get_traced_memory()
    memory_peak /= 1024*1024
    mem_peaks_mb.append(memory_peak)
    tracemalloc.stop()

mean_mem = sum(mem_peaks_mb) / len(mem_peaks_mb)
dev_mem = sqrt(sum([(x - mean_mem)**2 for x in mem_peaks_mb]) / (len(mem_peaks_mb)-1))
print(f"Promedio pico de memoria ocupada para búsqueda primero en profundidad: {round(mean_mem, 2)} [MB] en 10 ejecuciones. Desvío estándar: {round(dev_mem, 2)}.", )

mean_time = sum(time_measured) / len(time_measured)
dev_time = sqrt(sum([(x - mean_time)**2 for x in time_measured]) / (len(time_measured)-1))
print(f"Promedio tiempo de ejecución para búsqueda primero en profundidad: {round(mean_time, 2)} [s] en 10 ejecuciones. Desvío estándar: {round(dev_time, 2)}.", )

Promedio pico de memoria ocupada para búsqueda primero en profundidad: 0.23 [MB] en 10 ejecuciones. Desvío estándar: 0.04.
Promedio tiempo de ejecución para búsqueda primero en profundidad: 0.03 [s] en 10 ejecuciones. Desvío estándar: 0.01.


In [7]:
import tracemalloc
from math import sqrt
import time

mem_peaks_mb = []
time_measured = []

for _ in range(0, 10):
    tracemalloc.start()
    start_time = time.time()

    greedy_search(number_disks=num_disks)
    time_measured.append(time.time() - start_time)

    # Para medir memoria consumida usamos el pico de memoria
    _, memory_peak = tracemalloc.get_traced_memory()
    memory_peak /= 1024*1024
    mem_peaks_mb.append(memory_peak)
    tracemalloc.stop()

mean_mem = sum(mem_peaks_mb) / len(mem_peaks_mb)
dev_mem = sqrt(sum([(x - mean_mem)**2 for x in mem_peaks_mb]) / (len(mem_peaks_mb)-1))
print(f"Promedio pico de memoria ocupada para búsqueda greedy: {round(mean_mem, 2)} [MB] en 10 ejecuciones. Desvío estándar: {round(dev_mem, 2)}.", )

mean_time = sum(time_measured) / len(time_measured)
dev_time = sqrt(sum([(x - mean_time)**2 for x in time_measured]) / (len(time_measured)-1))
print(f"Promedio tiempo de ejecución para búsqueda greedy: {round(mean_time, 2)} [s] en 10 ejecuciones. Desvío estándar: {round(dev_time, 2)}.", )

Promedio pico de memoria ocupada para búsqueda greedy: 0.23 [MB] en 10 ejecuciones. Desvío estándar: 0.1.
Promedio tiempo de ejecución para búsqueda greedy: 0.04 [s] en 10 ejecuciones. Desvío estándar: 0.0.


##### 7. Si la solución óptima es 2^k -1 movimientos con k igual al número de discos. Qué tan lejos está la solución del algoritmo implementado de esta solución óptima (se recomienda correr al menos 10 veces y usar el promedio de trayecto usado).


Solución óptima para 5 discos: 2^5 - 1 = 31 movimientos

1. Búsqueda primero en profundidad
   - La solución del algoritmo requiere 121 movimientos, muy lejos de la cantidad óptima de movimientos
     
2. Búsqueda Greedy
   - La solución del algoritmo requiere 31 movimientos, cantidad óptima de movimientos


#### Extra - Ejecución de simulación

In [8]:

# Archivos para simulacion

solution_depth_first.generate_solution_for_simulator(initial_state_file="./depth_first_initial_state.json", sequence_file="./depth_first_sequence.json")

solution_greedy.generate_solution_for_simulator(initial_state_file="./greedy_initial_state.json", sequence_file="./greedy_sequence.json")


Para visualizar simulación de "búsqueda en profundidad primero" ejecutar:  
`python3 ./simulation_depth_first_hanoi.py`

Para visualizar simulación de "búsqueda greedy" ejecutar:  
`python3 ./simulation_greedy_hanoi.py`